In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/traintest/train_summary.csv
/kaggle/input/traintest/test_summary.csv
/kaggle/input/settttt/mainediteddataset1.csv


In [2]:
data=pd.read_csv('/kaggle/input/settttt/mainediteddataset1.csv')
#data = data.loc[np.random.choice(data.index, size=3500)]
data.shape

(3500, 8)

In [14]:
#saving this dataset to get the same result further
#data.to_csv('multilabelsummarydataset.csv')

In [4]:
import nltk
#nltk.download()
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import re

import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [5]:

def cleanbasic(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', str(sentence))
    return cleantext


def cleanPunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    cleaned = cleaned.strip()
    cleaned = cleaned.replace("\n"," ")
    return cleaned


def keepAlpha(sentence):
    alpha_sent = ""
    for word in sentence.split():
        alpha_word = re.sub('[^a-z A-Z]+', ' ', word)
        alpha_sent += alpha_word
        alpha_sent += " "
    alpha_sent = alpha_sent.strip()
    return alpha_sent

In [6]:
data['Summary'] = data['Summary'].str.lower()
data['Summary'] = data['Summary'].apply(cleanbasic)
data['Summary'] = data['Summary'].apply(cleanPunc)
data['Summary'] = data['Summary'].apply(keepAlpha)
data.head()

,Title,Author/Publisher,Summary,Fantasy,Fiction,Novel,Science Fiction,Speculative fiction
4164,Wolf's Blood,Jane Lindskold,it is revealed here that blind seer is a once ...,1,0,0,0,0
2454,Checkpoint: A Novel,Nicholson Baker,the main characters are two men jay and ben th...,0,1,0,0,0
1817,Preserver,Judith Reeves-Stevens,kirks mirror universe double has risen to powe...,0,1,0,1,1
3346,The Wizard of London,Mercedes Lackey,as an acting prequel to the elemental masters ...,1,0,0,0,1
3107,Rogue Ship,A. E. van Vogt,rogue ship is a standalone novel in a form of ...,0,0,0,1,1


In [7]:
stop_words = set(stopwords.words('english'))
stop_words.update(['zero','one','two','three','four','five','six','seven','eight','nine','ten','may','also','across','among','beside','however','yet','within'])
re_stop_words = re.compile(r"\b(" + "|".join(stop_words) + ")\\W", re.I)
def removeStopWords(sentence):
    global re_stop_words
    return re_stop_words.sub(" ", sentence)

data['Summary'] = data['Summary'].apply(removeStopWords)
data.head()

,Title,Author/Publisher,Summary,Fantasy,Fiction,Novel,Science Fiction,Speculative fiction
4164,Wolf's Blood,Jane Lindskold,revealed blind seer dead sorcerer exp...,1,0,0,0,0
2454,Checkpoint: A Novel,Nicholson Baker,main characters men jay ben novel consist...,0,1,0,0,0
1817,Preserver,Judith Reeves-Stevens,kirks mirror universe double risen power e...,0,1,0,1,1
3346,The Wizard of London,Mercedes Lackey,acting prequel elemental masters series b...,1,0,0,0,1
3107,Rogue Ship,A. E. van Vogt,rogue ship standalone novel form fixup ...,0,0,0,1,1


In [8]:
stemmer = SnowballStemmer("english")
def stemming(sentence):
    stemSentence = ""
    for word in sentence.split():
        stem = stemmer.stem(word)
        stemSentence += stem
        stemSentence += " "
    stemSentence = stemSentence.strip()
    return stemSentence

data['Summary'] = data['Summary'].apply(stemming)
data.head()

,Title,Author/Publisher,Summary,Fantasy,Fiction,Novel,Science Fiction,Speculative fiction
4164,Wolf's Blood,Jane Lindskold,reveal blind seer dead sorcer explain blue eye...,1,0,0,0,0
2454,Checkpoint: A Novel,Nicholson Baker,main charact men jay ben novel consist dialogu...,0,1,0,0,0
1817,Preserver,Judith Reeves-Stevens,kirk mirror univers doubl risen power evil emp...,0,1,0,1,1
3346,The Wizard of London,Mercedes Lackey,act prequel element master seri book detail pa...,1,0,0,0,1
3107,Rogue Ship,A. E. van Vogt,rogu ship standalon novel form fixup form shor...,0,0,0,1,1


In [9]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, random_state=42, test_size=0.20, shuffle=True)

In [10]:
train_text = train['Summary']
test_text = test['Summary']

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', ngram_range=(1,3), norm='l2')
vectorizer.fit(train_text)
vectorizer.fit(test_text)

TfidfVectorizer(ngram_range=(1, 3), strip_accents='unicode')

In [12]:
x_train = vectorizer.transform(train_text)
y_train = train.drop(labels = ['Title','Author/Publisher','Summary'], axis=1)
y_train = y_train.apply(pd.to_numeric,errors='ignore')

x_test = vectorizer.transform(test_text)
y_test = test.drop(labels = ['Title','Author/Publisher','Summary'], axis=1)
y_test = y_test.apply(pd.to_numeric,errors='ignore')


In [13]:

# using binary relevance
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# initialize binary relevance multi-label classifier
# with a gaussian naive bayes base classifier
classifier = BinaryRelevance(GaussianNB())

# train
classifier.fit(x_train, y_train)

# predict
predictions = classifier.predict(x_test)

# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))
print("\n")

Accuracy =  0.5785714285714286




In [15]:
from sklearn.linear_model import LogisticRegression

In [16]:
# initialize binary relevance multi-label classifier
# with a gaussian naive bayes base classifier
classifier = BinaryRelevance(LogisticRegression())

# train
classifier.fit(x_train, y_train)

# predict
predictions = classifier.predict(x_test)

# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))
print("\n")

Accuracy =  0.31142857142857144




In [ ]:
# using classifier chains
from skmultilearn.problem_transform import ClassifierChain

# initialize classifier chains multi-label classifier
classifier = ClassifierChain(LogisticRegression())

# Training logistic regression model on train data
classifier.fit(x_train, y_train)

# predict
predictions = classifier.predict(x_test)

# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))
print("\n")

In [20]:

# initialize classifier chains multi-label classifier
classifier = ClassifierChain(GaussianNB())

# Training logistic regression model on train data
classifier.fit(x_train, y_train)

# predict
predictions = classifier.predict(x_test)

# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))
print("\n")

Accuracy =  0.5771428571428572




In [21]:
# using Label Powerset
from skmultilearn.problem_transform import LabelPowerset
from sklearn.metrics import accuracy_score

In [22]:
%%time

# initialize label powerset multi-label classifier
classifier = LabelPowerset(LogisticRegression())

# train
classifier.fit(x_train, y_train)

# predict
predictions = classifier.predict(x_test)

# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))
print("\n")

Accuracy =  0.5157142857142857


CPU times: user 11min 12s, sys: 19.1 s, total: 11min 31s
Wall time: 6min 24s


In [23]:
%%time

# initialize label powerset multi-label classifier
classifier = LabelPowerset(GaussianNB())

# train
classifier.fit(x_train, y_train)

# predict
predictions = classifier.predict(x_test)

# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))
print("\n")

Accuracy =  0.6114285714285714


CPU times: user 46.1 s, sys: 33.1 s, total: 1min 19s
Wall time: 1min 19s


In [14]:

# http://scikit.ml/api/api/skmultilearn.adapt.html#skmultilearn.adapt.MLkNN

from skmultilearn.adapt import MLkNN
from scipy.sparse import csr_matrix, lil_matrix

In [15]:
%%time

classifier_new = MLkNN(k=10)

# Note that this classifier can throw up errors when handling sparse matrices.

x_train = lil_matrix(x_train).toarray()
y_train = lil_matrix(y_train).toarray()
x_test = lil_matrix(x_test).toarray()

# train
classifier_new.fit(x_train, y_train)

# predict
predictions_new = classifier_new.predict(x_test)

# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions_new))
print("\n")

Accuracy =  0.3385714285714286


CPU times: user 1h 46min 28s, sys: 3.93 s, total: 1h 46min 32s
Wall time: 1h 46min 34s


In [9]:
train.shape
train.to_csv('train_summary.csv',index=False)

In [10]:
test.shape
test.to_csv('test_summary.csv',index=False)